In [1]:
import numpy as np
import cv2
from pathlib import Path
import sys
sys.path.append('../src')

In [2]:
TEST_IMAGE_PATH = Path('../data/given/clear/33574917403_d4a68d6f52_c.jpg')
TEST_DEPTH_PATH = Path(str(TEST_IMAGE_PATH.with_suffix('')) + '_depth.png')

In [ ]:
from matplotlib import pyplot as plt

image = cv2.imread(TEST_IMAGE_PATH)
depth = cv2.imread(TEST_DEPTH_PATH, cv2.IMREAD_GRAYSCALE)
depth.resize(image.shape[:2])
depth = depth[..., np.newaxis] / 255.0

plt.figure(figsize=(12, 6))

plt.subplot(121)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title('Original Image')
plt.axis('off')

plt.subplot(122)
plt.imshow(depth, cmap='gray')
plt.colorbar()
plt.title('Depth Map')
plt.axis('off')

plt.tight_layout()
plt.show()


In [4]:
brightest_point = np.unravel_index(np.argmax(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)), image.shape[:2])
c_inf = image[brightest_point]
print(f"Brightest point coordinates (y, x): {brightest_point}, {c_inf}")


Brightest point coordinates (y, x): (np.int64(128), np.int64(503)), [251 255 255]


In [ ]:
from dcp_dehaze import haze, dehaze

hazed = haze(image, depth, 2)
cv2.imwrite('hazed.jpg', hazed)
dehazed = dehaze(hazed, dcp_kernel_size=31, filter_size=31, t0=0.1, dcp_kernel_type='square')
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(hazed, cv2.COLOR_BGR2RGB))
plt.title('Hazed Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor((dehazed), cv2.COLOR_BGR2RGB))
plt.title('Dehazed Image')
plt.axis('off')

plt.tight_layout()


In [10]:
from tqdm import tqdm
from PIL import Image

comparsion_frames = []
for degree in tqdm(np.arange(0.5, 15, 0.05)):
    hazed = haze(image, depth, degree)
    dehazed = dehaze(hazed, dcp_kernel_size=31, filter_size=31, t0=0.05)
    concatenated = np.concatenate((hazed, dehazed), axis=1)
    
    pil_image = Image.fromarray(cv2.cvtColor(concatenated, cv2.COLOR_BGR2RGB))
    comparsion_frames.append(pil_image)

100%|██████████| 290/290 [00:44<00:00,  6.51it/s]


In [ ]:
from IPython.display import Image as IPImage
# Save as GIF
print("Creating a GIF...")
comparsion_frames[0].save('comparsion.gif', save_all=True, append_images=comparsion_frames[1:], duration=50, loop=0)
IPImage(filename='comparsion.gif')